# Python для анализа данных

## API


## API YouTube

+ [Инструкция](https://sphenoid-aluminum-1ba.notion.site/API-YouTube-a1e2552472c246d295b6f1d1dbce5dcb) по получению ключа API YouTube
+ Официальная [документация](https://developers.google.com/youtube/v3/docs) к API YouTube
+ Статья на Хабр. [Что такое API?](https://habr.com/ru/post/464261/)
+ Статья на Хабр. [Что такое JSON](https://habr.com/ru/post/554274/)

API (Application programming interface) — это контракт, который предоставляет программа. Иными словами, это функции от разработчика, которые предоставлены в наше использование и распоряжение.

Сегодня попробуем потыкаться в API YouTube и вытащить какую-нибудь информацию. Но для этого нам нужен собственно доступ к API (ключ). Как его получить, можно узнать в [инструкции](https://sphenoid-aluminum-1ba.notion.site/API-YouTube-a1e2552472c246d295b6f1d1dbce5dcb).

In [1]:
from config import KEY_Y

In [3]:
!pip install google-api-python-client

     |████████████████████████████████| 12.8 MB 10.1 MB/s eta 0:00:01    |██████████████████████▎         | 8.9 MB 10.1 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 8.1 MB/s  eta 0:00:01
     |████████████████████████████████| 122 kB 7.8 MB/s eta 0:00:01
     |████████████████████████████████| 230 kB 8.3 MB/s eta 0:00:01
  Attempting uninstall: googleapis-common-protos
    Found existing installation: googleapis-common-protos 1.53.0
    Uninstalling googleapis-common-protos-1.53.0:
      Successfully uninstalled googleapis-common-protos-1.53.0
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.25.1
    Uninstalling google-api-core-1.25.1:
      Successfully uninstalled google-api-core-1.25.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-core 1.7.1 requires google-api-core<2.0.0dev,>=1.21.0

In [4]:
import googleapiclient.discovery as api

In [5]:
# key = 'ВАШ КЛЮЧ'
youtube = api.build('youtube', 'v3', developerKey = KEY_Y)

In [6]:
youtube

Далее используем запрос, который предлагается нам в самой документации. Этот тот момент, где вам не нужно задумываться о том, почему метод называется так и почему он хочет определенные параметры — просто так придумал разработчик.

### Видео по опеределенному запросу

Например, в самой [документации](https://developers.google.com/youtube/v3/docs/search/list?hl=en) про параметр `part` сказано Set the parameter value to snippet.

`maxResults` — сколько видео (максимально можно 50, но, спойлер, это ограничение можно обойти) \
`q` — запрос, который мы ищем \
`type` — пока мы ищем видео, но можно и каналы \
`order` — по релевантности (его можно убрать) \
Это все наш запрос с помощью метода .`search()`. Выгружаем его командой `.execute()`.

In [7]:
request_search = youtube.search().list(
    part = "snippet",
    maxResults = 10,
    q = "API with Python",
    type = 'video',
    order = 'relevance')

response_search = request_search.execute()
response_search

{'kind': 'youtube#searchListResponse',
 'etag': 'cGqlevWICgL4130HRh4JmC9LW-w',
 'nextPageToken': 'CAoQAA',
 'regionCode': 'RU',
 'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 10},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'Gj0xrUH-6U2pMjdt23qUAv0igJE',
   'id': {'kind': 'youtube#video', 'videoId': '0sOvCWFmrtA'},
   'snippet': {'publishedAt': '2021-11-01T13:11:23Z',
    'channelId': 'UC8butISFwT-Wl7EV0hUK0BQ',
    'title': 'Python API Development - Comprehensive Course for Beginners',
    'description': 'Learn Python API development in one of the most comprehensive courses ever on the topic. You will build a full-fledged API in ...',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/0sOvCWFmrtA/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/0sOvCWFmrtA/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/0sOvCWFmrtA/hqdefault.jpg',
      'width': 4

Это просто словарь! А с ними вы работать уже умеете.

К слову, использование этого API **бесплатно**, но не бесконечно. Есть определенный лимит для одного ключа —10.000 у.е в день (например, запрос видео выше стоил нам 100 у.е). Подробнее об этом можно почитать в самой документации. Такое придумано в API YouTube (для других API может быть что-то свое или вообще не быть лимита).

Посмотрим, какие тут есть ключи:

In [8]:
response_search.keys()

dict_keys(['kind', 'etag', 'nextPageToken', 'regionCode', 'pageInfo', 'items'])

Из них нас интересуют только `nextPageToken`, `pageInfo`, `items`.

`nextPageToken` — адрес следующей страницы нашего поиска; до тех пор пока он есть в выдаче, мы можем собирать данные. \
`pageInfo` — сколько всего результатов и сколько всего результатов по нашему запросу. Если всего результатов очень много, то все вы не получите (так уж устроен API YouTube); можно исхитриться и в цикле делать запрос от и до определенной даты. \
`items` — все видео, которые мы нашли (у нас их десять).

In [9]:
print(type(response_search['items'])) # это список! 
print(len(response_search['items']))  # наши десять видео

<class 'list'>
10


Вытащим самое первое видео и посмотрим, что там.

In [10]:
response_search['items'][0]

{'kind': 'youtube#searchResult',
 'etag': 'Gj0xrUH-6U2pMjdt23qUAv0igJE',
 'id': {'kind': 'youtube#video', 'videoId': '0sOvCWFmrtA'},
 'snippet': {'publishedAt': '2021-11-01T13:11:23Z',
  'channelId': 'UC8butISFwT-Wl7EV0hUK0BQ',
  'title': 'Python API Development - Comprehensive Course for Beginners',
  'description': 'Learn Python API development in one of the most comprehensive courses ever on the topic. You will build a full-fledged API in ...',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/0sOvCWFmrtA/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/0sOvCWFmrtA/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/0sOvCWFmrtA/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'freeCodeCamp.org',
  'liveBroadcastContent': 'none',
  'publishTime': '2021-11-01T13:11:23Z'}}

In [11]:
response_search['items'][0].keys()

dict_keys(['kind', 'etag', 'id', 'snippet'])

Тут нас интересуют ключи `id` и `snippet`.

* в `id` записано `'videoId'` (оно нам нужно, чтобы вытащить информацию о конкретном видео, например, статистику по лайкам или комментарии)
* `snippet` содержит основную информацию о видео (id канала, заголовок, описание).

In [16]:
response_search['items'][0]['id']['videoId']

'zsYIw6RXjfM'

К слову, само видео можно посмотреть так: https://www.youtube.com/watch?v=zsYIw6RXjfM. Обратите внимание, куда именно подставлятся наше id.

## Комментарии к определенному видео

Например, можно забрать комментарии определенного видео (а в перспективе — посмотреть их тональность). Поможет нам в этом метод `.commentThreads()`.

In [12]:
comment_request = youtube.commentThreads().list(
    part = 'snippet',
    videoId = "zsYIw6RXjfM"
)

comment_response = comment_request.execute()
comment_response

{'kind': 'youtube#commentThreadListResponse',
 'etag': 'DMF4-6cnGTazNPXbWB3jDJ1d1_c',
 'nextPageToken': 'Z2V0X25ld2VzdF9maXJzdC0tQ2dnSWdBUVZGN2ZST0JJRkNKMGdHQUVTQlFpSklCZ0FFZ1VJaUNBWUFCSUZDSWNnR0FBWUFDSU9DZ3dJNHFENXBnWVFrTXVROGdJ',
 'pageInfo': {'totalResults': 20, 'resultsPerPage': 20},
 'items': [{'kind': 'youtube#commentThread',
   'etag': '_dbl6xi43lVMHR4fF4Lf37eY4G4',
   'id': 'Ugwv2of8WoJe24fHanB4AaABAg',
   'snippet': {'channelId': 'UC4JX40jDee_tINbkjycV4Sg',
    'videoId': 'zsYIw6RXjfM',
    'topLevelComment': {'kind': 'youtube#comment',
     'etag': 'NGYcuZOzdo8j8UK2maS15t44tSs',
     'id': 'Ugwv2of8WoJe24fHanB4AaABAg',
     'snippet': {'channelId': 'UC4JX40jDee_tINbkjycV4Sg',
      'videoId': 'zsYIw6RXjfM',
      'textDisplay': 'Rapidly build and deploy APIs in seconds with DreamFactory ! <a href="https://bit.ly/3M1TAP7">https://bit.ly/3M1TAP7</a>',
      'textOriginal': 'Rapidly build and deploy APIs in seconds with DreamFactory ! https://bit.ly/3M1TAP7',
      'authorDispla

In [13]:
for comment in comment_response['items']:
    print(comment['snippet']['topLevelComment']['snippet']['textOriginal'], '\n')

Rapidly build and deploy APIs in seconds with DreamFactory ! https://bit.ly/3M1TAP7 

What keyboard do you have? 

The delivery of the information was very clear, but I had difficulty understanding the "why" behind a lot of things you did in the video. 

Youre superior for this unimaginable website hosting video! Ive experimented with Hostwinds and HostGator, however Cloudways with TST20 coupon is the best money saver. 

Youre superior for this unimaginable website hosting video! Ive experimented with Hostwinds and HostGator, however Cloudways with TST20 coupon is the best money saver. 

Cool 

Thanks! 

No idea what you did there, since it's my first time using Flask, but it works. Thank you 

"/get-user", "/create-user" 😂😂😂😂😂😂😂😂 

Hi, Can requests be used to test and not postman?? 

Thank you so much. I like yours tutorials 

Hey Tim, in this video you looked at creating APIs with Flask and the way it is done is mostly with functions and decorators but at the end of the video there w

## Статистика по определенному видео
Последний метод в этом погружении в API YouTube — `.videos()`

In [14]:
video_request = youtube.videos().list(
    part = ['snippet', 'contentDetails', 'id', 'statistics'],
    id = 'zsYIw6RXjfM'
)

video_response = video_request.execute()
video_response

{'kind': 'youtube#videoListResponse',
 'etag': 'OOqbvMSW-TK3Sozb9MjCAzQpUG4',
 'items': [{'kind': 'youtube#video',
   'etag': 'eeTmXwz4XRKSf8xwy6fflddAKlg',
   'id': 'zsYIw6RXjfM',
   'snippet': {'publishedAt': '2023-05-23T13:00:53Z',
    'channelId': 'UC4JX40jDee_tINbkjycV4Sg',
    'title': 'Create A Python API in 12 Minutes',
    'description': 'Rapidly build and deploy APIs in seconds with DreamFactory ! https://bit.ly/3M1TAP7\n\nAre you learning programming and wondering what an API is? An API is an Application Programming Interface that allows different software or systems to communicate with one another. I\'ll show you how to whip up your very own API in just 10 minutes using Python and Flask. \n\nPost Man API Platform: \nhttps://www.postman.com/\n\nFix Pip (Windows): \nhttps://www.youtube.com/watch?v=AdUZArA-kZw&t=204s\n\nFix Pip (Mac): \nhttps://www.youtube.com/watch?v=E-WhAS6qzsU\n\n💻 Master Blockchain and Web 3.0 development today by using BlockchainExpert: 🔗 https://algoexpe

Теперь попробуем собрать информацию о всех наших десяти видео!

In [15]:
video_ids = []

for video in response_search['items']:
    video_ids.append(video['id']['videoId'])
    
print(video_ids)

['0sOvCWFmrtA', 'zsYIw6RXjfM', 'qbLc5a9jdXo', '0x0hOdvhCXU', 'fXa_2rllZTI', 'VfkTcR4J3Y4', '-ykeT6kk4bk', '2HfSFdPEFRg', 'WXsD0ZgxjRw', 'hpc5jyVpUpw']


In [16]:
video_10_request = youtube.videos().list(
    part = ['snippet', 'contentDetails', 'id', 'statistics'],
    id = video_ids
)

video_10_response = video_10_request.execute()

In [17]:
video_10_response

{'kind': 'youtube#videoListResponse',
 'etag': 'ZHOgoVwLJCuKjBKmFQ19RxNGbXk',
 'items': [{'kind': 'youtube#video',
   'etag': 'ZAqOvcycDHJTeNQzPlj2SvZZ14E',
   'id': '0sOvCWFmrtA',
   'snippet': {'publishedAt': '2021-11-01T13:11:23Z',
    'channelId': 'UC8butISFwT-Wl7EV0hUK0BQ',
    'title': 'Python API Development - Comprehensive Course for Beginners',
    'description': 'Learn Python API development in one of the most comprehensive courses ever on the topic. You will build a full-fledged API in Python using FastAPI. You will learn the fundamentals of API design including routes, serialization/deserialization, schema validation, and models. You will also learn about SQL, testing with pytest, and how to build out a CI/CD pipeline using GitHub actions.\n\n💻 Code: https://github.com/Sanjeev-Thiyagarajan/fastapi-course\n\n✏️ Course from Sanjeev Thiyagarajan. Check out his channel: https://www.youtube.com/channel/UC2sYgV-NV6S5_-pqLGChoNQ\n\n⭐️ Course Contents\n00:00 Intro\n06:33 Project Ov

## Базовая обработка выгрузки из API

Чтобы привести это все в относительно приличный вид, сделаем таблицу. Поможет нам в этом модуль pandas, нам понадобится только один метод — `json_normalize()`.

In [18]:
import pandas as pd
table = pd.json_normalize(video_10_response['items'])
table

,kind,etag,id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,...,contentDetails.dimension,contentDetails.definition,contentDetails.caption,contentDetails.licensedContent,contentDetails.projection,statistics.viewCount,statistics.likeCount,statistics.favoriteCount,statistics.commentCount,snippet.tags
0,youtube#video,ZAqOvcycDHJTeNQzPlj2SvZZ14E,0sOvCWFmrtA,2021-11-01T13:11:23Z,UC8butISFwT-Wl7EV0hUK0BQ,Python API Development - Comprehensive Course ...,Learn Python API development in one of the mos...,https://i.ytimg.com/vi/0sOvCWFmrtA/default.jpg,120,90,...,2d,hd,true,True,rectangular,1808540,46984,0,1897,NaN
1,youtube#video,eeTmXwz4XRKSf8xwy6fflddAKlg,zsYIw6RXjfM,2023-05-23T13:00:53Z,UC4JX40jDee_tINbkjycV4Sg,Create A Python API in 12 Minutes,Rapidly build and deploy APIs in seconds with ...,https://i.ytimg.com/vi/zsYIw6RXjfM/default.jpg,120,90,...,2d,hd,true,True,rectangular,309748,7218,0,96,"[tech with tim, build your first api in 10 min..."
2,youtube#video,Z0LPjn6oqjtPD4NzsJsV1O2tQ-0,qbLc5a9jdXo,2020-11-19T20:27:58Z,UCZUyPT9DkJWmS_DzdOi7RIA,REST API Crash Course - Introduction + Full Py...,Start your software dev career - https://calcu...,https://i.ytimg.com/vi/qbLc5a9jdXo/default.jpg,120,90,...,2d,hd,false,True,rectangular,775075,18736,0,634,"[rest api, rest apis, python api, api introduc..."
3,youtube#video,CIkDHjK17N_ALxbK2wBs2yIULhs,0x0hOdvhCXU,2019-10-10T07:52:52Z,UCGuhp4lpQvK94ZC5kuOZbjA,Урок 4. Python. Быстрый старт. Изучаем API сер...,"Подробнее о курсе ""Python. Полное руководство""...",https://i.ytimg.com/vi/0x0hOdvhCXU/default.jpg,120,90,...,2d,hd,false,False,rectangular,63186,1026,0,26,"[Python, уроки по python, уроки по python 3.7,..."
4,youtube#video,KwvNKHoMuHcC9U9tbRQIWuTYw8A,fXa_2rllZTI,2022-06-02T13:45:01Z,UCSTYGpIpMIiQPspjLplB6Ow,Что такое API?,👉 Получи высокооплачиваемые знания в Merion Ac...,https://i.ytimg.com/vi/fXa_2rllZTI/default.jpg,120,90,...,2d,hd,true,False,rectangular,251280,13079,0,313,"[api, Application Programming Interface, REST ..."
5,youtube#video,wxvu3fJ-6TASfVWd578Zcr3eh1s,VfkTcR4J3Y4,2022-03-25T14:00:14Z,UCvuY904el7JvBlPbdqbfguw,Python REST API. Разработка RESTFul проекта на...,Представляю вашему вниманию большой урок по из...,https://i.ytimg.com/vi/VfkTcR4J3Y4/default.jpg,120,90,...,2d,hd,false,True,rectangular,56474,1514,0,88,"[rest api python, rest api, restful api, pytho..."
6,youtube#video,ajW__n3eMkwT0JVaaLN0VfKO8EU,-ykeT6kk4bk,2021-05-29T15:00:00Z,UC4JX40jDee_tINbkjycV4Sg,Python FAST API Tutorial,"Welcome back to my channel! In this video, I w...",https://i.ytimg.com/vi/-ykeT6kk4bk/default.jpg,120,90,...,2d,hd,true,True,rectangular,280855,7315,0,251,"[tech with tim, tech with tim tutorial, fast a..."
7,youtube#video,jtqOw5qiE4WkDysNVEML8hDv2PU,2HfSFdPEFRg,2022-05-31T10:30:01Z,UC7cs8q-gJRlGwj4A8OmCmXg,How to use a Public API | Using a Public API w...,APIs are extremely useful to know how to use. ...,https://i.ytimg.com/vi/2HfSFdPEFRg/default.jpg,120,90,...,2d,hd,false,True,rectangular,23200,906,0,58,"[Data Analyst, Data Analyst job, Data Analyst ..."
8,youtube#video,LAyUpvGRshnY_Uk2M61CplkJBMs,WXsD0ZgxjRw,2023-02-06T15:53:28Z,UC8butISFwT-Wl7EV0hUK0BQ,APIs for Beginners 2023 - How to use an API (F...,What is an API? Learn all about APIs (Applicat...,https://i.ytimg.com/vi/WXsD0ZgxjRw/default.jpg,120,90,...,2d,hd,true,True,rectangular,1118517,24309,0,534,NaN
9,youtube#video,Sj0hfvqjn5NmKHQqPybws5lj98E,hpc5jyVpUpw,2019-11-02T14:00:09Z,UCF1dnm4HfA9LYQo9fVU5RKQ,Working with APIs in Python - Code in 10 Minutes,"In this video, I teach you the basics of APIs ...",https://i.ytimg.com/vi/hpc5jyVpUpw/default.jpg,120,90,...,2d,hd,false,True,rectangular,277122,4738,0,113,"[programming, education, course, python, wordp..."


In [19]:
table['statistics.viewCount'] = pd.to_numeric(table['statistics.viewCount'])

In [20]:
table.sort_values(by = 'statistics.viewCount', ascending = False)

,kind,etag,id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,...,contentDetails.dimension,contentDetails.definition,contentDetails.caption,contentDetails.licensedContent,contentDetails.projection,statistics.viewCount,statistics.likeCount,statistics.favoriteCount,statistics.commentCount,snippet.tags
0,youtube#video,ZAqOvcycDHJTeNQzPlj2SvZZ14E,0sOvCWFmrtA,2021-11-01T13:11:23Z,UC8butISFwT-Wl7EV0hUK0BQ,Python API Development - Comprehensive Course ...,Learn Python API development in one of the mos...,https://i.ytimg.com/vi/0sOvCWFmrtA/default.jpg,120,90,...,2d,hd,true,True,rectangular,1808540,46984,0,1897,NaN
8,youtube#video,LAyUpvGRshnY_Uk2M61CplkJBMs,WXsD0ZgxjRw,2023-02-06T15:53:28Z,UC8butISFwT-Wl7EV0hUK0BQ,APIs for Beginners 2023 - How to use an API (F...,What is an API? Learn all about APIs (Applicat...,https://i.ytimg.com/vi/WXsD0ZgxjRw/default.jpg,120,90,...,2d,hd,true,True,rectangular,1118517,24309,0,534,NaN
2,youtube#video,Z0LPjn6oqjtPD4NzsJsV1O2tQ-0,qbLc5a9jdXo,2020-11-19T20:27:58Z,UCZUyPT9DkJWmS_DzdOi7RIA,REST API Crash Course - Introduction + Full Py...,Start your software dev career - https://calcu...,https://i.ytimg.com/vi/qbLc5a9jdXo/default.jpg,120,90,...,2d,hd,false,True,rectangular,775075,18736,0,634,"[rest api, rest apis, python api, api introduc..."
1,youtube#video,eeTmXwz4XRKSf8xwy6fflddAKlg,zsYIw6RXjfM,2023-05-23T13:00:53Z,UC4JX40jDee_tINbkjycV4Sg,Create A Python API in 12 Minutes,Rapidly build and deploy APIs in seconds with ...,https://i.ytimg.com/vi/zsYIw6RXjfM/default.jpg,120,90,...,2d,hd,true,True,rectangular,309748,7218,0,96,"[tech with tim, build your first api in 10 min..."
6,youtube#video,ajW__n3eMkwT0JVaaLN0VfKO8EU,-ykeT6kk4bk,2021-05-29T15:00:00Z,UC4JX40jDee_tINbkjycV4Sg,Python FAST API Tutorial,"Welcome back to my channel! In this video, I w...",https://i.ytimg.com/vi/-ykeT6kk4bk/default.jpg,120,90,...,2d,hd,true,True,rectangular,280855,7315,0,251,"[tech with tim, tech with tim tutorial, fast a..."
9,youtube#video,Sj0hfvqjn5NmKHQqPybws5lj98E,hpc5jyVpUpw,2019-11-02T14:00:09Z,UCF1dnm4HfA9LYQo9fVU5RKQ,Working with APIs in Python - Code in 10 Minutes,"In this video, I teach you the basics of APIs ...",https://i.ytimg.com/vi/hpc5jyVpUpw/default.jpg,120,90,...,2d,hd,false,True,rectangular,277122,4738,0,113,"[programming, education, course, python, wordp..."
4,youtube#video,KwvNKHoMuHcC9U9tbRQIWuTYw8A,fXa_2rllZTI,2022-06-02T13:45:01Z,UCSTYGpIpMIiQPspjLplB6Ow,Что такое API?,👉 Получи высокооплачиваемые знания в Merion Ac...,https://i.ytimg.com/vi/fXa_2rllZTI/default.jpg,120,90,...,2d,hd,true,False,rectangular,251280,13079,0,313,"[api, Application Programming Interface, REST ..."
3,youtube#video,CIkDHjK17N_ALxbK2wBs2yIULhs,0x0hOdvhCXU,2019-10-10T07:52:52Z,UCGuhp4lpQvK94ZC5kuOZbjA,Урок 4. Python. Быстрый старт. Изучаем API сер...,"Подробнее о курсе ""Python. Полное руководство""...",https://i.ytimg.com/vi/0x0hOdvhCXU/default.jpg,120,90,...,2d,hd,false,False,rectangular,63186,1026,0,26,"[Python, уроки по python, уроки по python 3.7,..."
5,youtube#video,wxvu3fJ-6TASfVWd578Zcr3eh1s,VfkTcR4J3Y4,2022-03-25T14:00:14Z,UCvuY904el7JvBlPbdqbfguw,Python REST API. Разработка RESTFul проекта на...,Представляю вашему вниманию большой урок по из...,https://i.ytimg.com/vi/VfkTcR4J3Y4/default.jpg,120,90,...,2d,hd,false,True,rectangular,56474,1514,0,88,"[rest api python, rest api, restful api, pytho..."
7,youtube#video,jtqOw5qiE4WkDysNVEML8hDv2PU,2HfSFdPEFRg,2022-05-31T10:30:01Z,UC7cs8q-gJRlGwj4A8OmCmXg,How to use a Public API | Using a Public API w...,APIs are extremely useful to know how to use. ...,https://i.ytimg.com/vi/2HfSFdPEFRg/default.jpg,120,90,...,2d,hd,false,True,rectangular,23200,906,0,58,"[Data Analyst, Data Analyst job, Data Analyst ..."
